In [ ]:
import numpy as np
import os
import torch
from neuropop import nn_prediction

# make small dataset
root = "/media/carsen/ssd2/rastermap_paper/"

device = torch.device("cpu")

dat = np.load(os.path.join(root, "data/", "spont_data.npz"))
spks, U, sv, V = dat["spks"], dat["U"], dat["sv"], dat["V"]
tcam, tneural = dat["tcam"], dat["tneural"]
beh = dat["beh"]

spks = spks[::60].astype("float32")
beh = beh.astype("float32")
U = U[::60]


In [ ]:

np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)

### fit nonlinear model from behavior to neural activity
Vfit = V.copy() * sv
pred_model = nn_prediction.PredictionNetwork(n_in=beh.shape[-1], n_kp=22, n_out=Vfit.shape[-1], )
y_pred_all, ve_all, itest = pred_model.train_model(beh, Vfit, tcam, tneural, delay=-1,
                                                        learning_rate=1e-3, n_iter=30,
                                                    device=device, verbose=True)
print(ve_all)


In [ ]:
### fit linear model from behavior to neural activity
### predict activity from behavior
from neuropop import linear_prediction
ve, _, spks_pred, itest = linear_prediction.prediction_wrapper(beh, spks.T, rank=32,
                                                                tcam=tcam, tneural=tneural,
                                                                lam=1e-2, delay=-1, device=device)
print(ve)

ve, _, spks_pred, itest = linear_prediction.prediction_wrapper(beh, spks.T, rank=None,
                                                                tcam=tcam, tneural=tneural,
                                                                lam=1e-2, delay=-1, device=device)
print(ve)

In [ ]:
from neuropop import dimensionality

scov, varcov = dimensionality.SVCA(spks)
alpha, py = dimensionality.get_powerlaw(scov, trange=np.arange(10,len(scov)))

print(alpha)

In [ ]:
np.savez("neuropop_test_data.npz", spks=spks, beh=beh, U=U, sv=sv, V=V, tcam=tcam, tneural=tneural)